<a href="https://colab.research.google.com/github/JoshiDeepak08/Machine_learning/blob/main/pipline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd


In [2]:
df= pd.read_csv("tested.csv")

In [3]:
df.sample(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
287,1179,0,1,"Snyder, Mr. John Pillsbury",male,24.0,1,0,21228,82.2667,B45,S
334,1226,0,3,"Cor, Mr. Ivan",male,27.0,0,0,349229,7.8958,NaN,S
416,1308,0,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S
326,1218,1,2,"Becker, Miss. Ruth Elizabeth",female,12.0,2,1,230136,39.0000,F4,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [5]:
df.drop(columns= ['PassengerId', 'Name', 'Cabin', 'Ticket'], inplace = True)

In [7]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,34.5,0,0,7.8292,Q
1,1,3,female,47.0,1,0,7.0000,S
2,0,2,male,62.0,0,0,9.6875,Q
3,0,3,male,27.0,0,0,8.6625,S
4,1,3,female,22.0,1,1,12.2875,S


In [9]:
df.isnull().sum()

,0
Survived,0
Pclass,0
Sex,0
Age,86
SibSp,0
Parch,0
Fare,1
Embarked,0


yahan pe survived column ko out maan kr saare steps m columns ki indexing ki gyi hai. (kyoki model ko milne wale training dataset m "survived" include nhi hoga.)

In [48]:
# 1--> simple imputer to impute values
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

tf1 = ColumnTransformer([
    ('impute_age', SimpleImputer(), [2]),
    ('impute_embarked',SimpleImputer(strategy= 'most_frequent'), [6])
], remainder='passthrough' )

In [49]:
#  2--> One Hot Encoding
tf2 = ColumnTransformer([
    ( 'ohe_sex_embarked', OneHotEncoder(sparse= False, handle_unknown= 'ignore'),[1,6] )
        ], remainder= 'passthrough')

In [50]:
#  3--> scaling the values
tf3 = ColumnTransformer([
    ('scale', MinMaxScaler(), slice(0,11))
])

In [58]:
#  feature selection
from sklearn.feature_selection import SelectKBest, chi2
tf4= SelectKBest( score_func= chi2, k= 11)

In [52]:
# model training
from sklearn.tree import DecisionTreeClassifier
tf5 = DecisionTreeClassifier()

**Create a Pipline:**

In [53]:
#  make pipeline using Make_pipeline instead of Pipeline function
from sklearn.pipeline import Pipeline, make_pipeline

pipe = make_pipeline( tf1, tf2, tf3, tf4, tf5)


In [54]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df.drop( columns =['Survived']), df['Survived'], test_size= 0.2, random_state= 42)

In [59]:
#  train the model
pipe.fit(x_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('columntransformer-2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('columntransformer-3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 11, None))])),
                ('selectkbest',
                 SelectKBest(score_func=<function chi2 at 0x789001115480>)),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [60]:
y_pred = pipe.predict( x_test)

In [61]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.5952380952380952